In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [28]:
# Chemin d'accès au fichier de données
file_path = 'data.csv'

# Réimporter le fichier CSV original pour recommencer à partir de zéro
data = pd.read_csv(file_path)

# Vérification des données manquantes et des doublons dans le DataFrame
missing_data = data.isnull().sum()  # Compte le nombre de valeurs manquantes par colonne
duplicates = data.duplicated().sum()  # Compte le nombre de lignes dupliquées
data = data.drop_duplicates()  # Supprime les lignes dupliquées

# Initialiser l'encodeur de labels pour la conversion des données catégorielles
label_encoder = LabelEncoder()

# Convertir les colonnes 'sex' et 'smoker' en valeurs entières
columns_to_encode = ['sex', 'smoker']
for column in columns_to_encode:
    data[column] = label_encoder.fit_transform(data[column])  # Encodage des colonnes

# Créer des variables binaires avec des noms explicites pour la colonne 'region'
region_dummies_named = pd.get_dummies(data['region'], prefix='is', prefix_sep='_')

# Concaténer ces variables binaires avec le DataFrame original
data_with_named_dummies = pd.concat([data, region_dummies_named], axis=1)


# Préparation de l'ensemble des caractéristiques (X) et de la variable cible (y)
X = data_with_named_dummies.drop('charges', axis=1)  # Ensemble des caractéristiques
y = data_with_named_dummies['charges']  # Variable cible

# Création d'une colonne binaire pour le statut de fumeur
X['smoker_binary'] = (X['smoker'] == 1).astype(int)

# Définition des intervalles pour les catégories de BMI
bins = [0, 18.5, 24.9, 29.9, 34.9, 39.9, float('inf')]  # Limites des catégories
labels = ['underweight', 'normal weight', 'overweight', 'obesity class I', 'obesity class II', 'obesity class III']

# Utilisation de pd.cut pour créer de nouvelles colonnes basées sur les catégories de BMI
X['BMI_category'] = pd.cut(X['bmi'], bins=bins, labels=labels, right=False)

# Utilisation de pd.get_dummies pour obtenir des colonnes binaires pour chaque catégorie de BMI
BMI_dummies = pd.get_dummies(X['BMI_category'])

# Ajout de ces colonnes binaires au DataFrame X
X = pd.concat([X, BMI_dummies], axis=1)

# Création d'une nouvelle caractéristique combinant BMI et statut de fumeur
X['bmi_smoker'] = X['bmi'] * X['smoker_binary']

# Suppression des colonnes temporaires
X = X.drop(['smoker_binary', 'BMI_category'], axis=1)

# Enregistrement du DataFrame modifié dans un nouveau fichier CSV
output_file_path = 'data.csv'
data_with_named_dummies.to_csv(output_file_path, index=False)

# Affichage des premières lignes des DataFrame finaux X et y
X.head(), y.head()


(   age  sex     bmi  children  smoker     region  is_northeast  is_northwest  \
 0   19    0  27.900         0       1  southwest         False         False   
 1   18    1  33.770         1       0  southeast         False         False   
 2   28    1  33.000         3       0  southeast         False         False   
 3   33    1  22.705         0       0  northwest         False          True   
 4   32    1  28.880         0       0  northwest         False          True   
 
    is_southeast  is_southwest  underweight  normal weight  overweight  \
 0         False          True        False          False        True   
 1          True         False        False          False       False   
 2          True         False        False          False       False   
 3         False         False        False           True       False   
 4         False         False        False          False        True   
 
    obesity class I  obesity class II  obesity class III  bmi_smok